参考代码：https://aistudio.baidu.com/aistudio/projectdetail/1894458?channelType=0&channel=0

#  0.PALM眼底彩照中黄斑中央凹定位比赛
比赛链接： [常规赛：PALM眼底彩照中黄斑中央凹定位](https://aistudio.baidu.com/aistudio/competition/detail/86)

## 0.0赛题介绍
PALM黄斑定位常规赛的重点是研究和发展与患者眼底照片黄斑结构定位相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上定位黄斑的自动算法。具体目的是预测黄斑中央凹在图像中的坐标值。

![](https://ai.bdstatic.com/file/D80ACC7A393348DD9F30518D57781D32)

## 0.1数据简介
PALM病理性近视预测常规赛由中山大学中山眼科中心提供800张带黄斑中央凹坐标标注的眼底彩照供选手训练模型，另提供400张带标注数据供平台进行模型测试。

## 0.2数据说明
本次常规赛提供的金标准由中山大学中山眼科中心的7名眼科医生手工进行标注，之后由另一位高级专家将它们融合为最终的标注结果。本比赛提供数据集对应的黄斑中央凹坐标信息存储在xlsx文件中，名为“Fovea_Location_train”，第一列对应眼底图像的文件名(包括扩展名“.jpg”)，第二列包含x坐标，第三列包含y坐标。

![](https://ai.bdstatic.com/file/1CD1DA54E68349CA8553678E80F4D40E)

## 0.3训练数据集
文件名称：Train
Train文件夹里有一个文件夹fundus_images和一个xlsx文件。

fundus_images文件夹内包含800张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、P0001.jpg、N0001.jpg和V0001.jpg。
xlsx文件中包含800张眼底彩照对应的x、y坐标信息。
## 0.4测试数据集
文件名称：PALM-Testing400-Images 文件夹里包含400张眼底彩照，命名形如T0001.jpg。

## 0.5提交格式
黄斑中央凹定位比赛的提交内容需将所有测试图像的黄斑坐标存入一个CSV文件，名为“Fovea_Localization_Results.csv”，第一列对应测试眼底图像的文件名(包括扩展名“.jpg”)，第二列包含x坐标，第三列包含y坐标。

![](https://ai.bdstatic.com/file/0D28C4FD24CA47748B867579C9A4CABC)

## 0.6思路
训练数据为图片以及中心点，要想从这些数据中推测测试数据中心店位置信息。可认为构造bbox，然后按目标检测来做，预测新的bbox，然后利用bbox来计算中心点，达到所需信息。

# 1.数据格式调整

In [ ]:
!unzip -aqo data/data85130/常规赛：PALM眼底彩照中黄斑中央凹定位.zip

In [ ]:
!rm _* -rf

In [ ]:
!mv '常规赛：PALM眼底彩照中黄斑中央凹定位' dataset

In [ ]:
!mv  dataset/Train/fundus_image  dataset/Train/JPEGImages 

In [ ]:
import pandas as pd

train_data=pd.read_excel('dataset/Train/Fovea_Location_train.xlsx')
train_data.head()


,imgName,Fovea_X,Fovea_Y
0,H0001.jpg,743.96,790.54
1,H0002.jpg,1394.82,725.54
2,H0003.jpg,1361.74,870.72
3,H0004.jpg,703.15,742.44
4,H0005.jpg,1070.95,1037.54


## 1.1 自定义生成VOC数据集格式函数

In [ ]:
# lxml操作需要用到
!pip install lxml

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 5.5MB 14.4MB/s eta 0:00:01


In [ ]:
from lxml.etree import Element, SubElement, tostring
from xml.dom.minidom import parseString
import os


def save_xml(image_name, bbox, save_dir='Annotations', width=10, height=10, channel=3):
    '''
    将CSV中的一行
    000000001.jpg [[1,2,3,4],...]
    转化成
    000000001.xml

    :param image_name:图片名
    :param bbox:对应的bbox
    :param save_dir:
    :param width:这个是图片的宽度，博主使用的数据集是固定的大小的，所以设置默认
    :param height:这个是图片的高度，博主使用的数据集是固定的大小的，所以设置默认
    :param channel:这个是图片的通道，博主使用的数据集是固定的大小的，所以设置默认
    :return:
    '''

    # 图片存放位置
    node_root = Element('annotation')
    node_folder = SubElement(node_root, 'folder')
    node_folder.text = 'JPEGImages'

    # 文件名
    node_filename = SubElement(node_root, 'filename')
    node_filename.text = os.path.join('JPEGImages',image_name.split('/')[-1])

    # 文件长
    node_size = SubElement(node_root, 'size')
    node_width = SubElement(node_size, 'width')
    node_width.text = '%s' % width

    # 文件高
    node_height = SubElement(node_size, 'height')
    node_height.text = '%s' % height

    # 文件通道
    node_depth = SubElement(node_size, 'depth')
    node_depth.text = '%s' % channel

    # bbox
    for x, y, w, h in bbox:
        print(x, y, w, h)
        left, top, right, bottom = x, y, x + w, y + h
        node_object = SubElement(node_root, 'object')
        node_name = SubElement(node_object, 'name')
        node_name.text = 'bad'
        node_difficult = SubElement(node_object, 'difficult')
        node_difficult.text = '0'
        node_bndbox = SubElement(node_object, 'bndbox')
        node_xmin = SubElement(node_bndbox, 'xmin')
        node_xmin.text = '%s' % left
        node_ymin = SubElement(node_bndbox, 'ymin')
        node_ymin.text = '%s' % top
        node_xmax = SubElement(node_bndbox, 'xmax')
        node_xmax.text = '%s' % right
        node_ymax = SubElement(node_bndbox, 'ymax')
        node_ymax.text = '%s' % bottom

    xml = tostring(node_root, pretty_print=True)
    dom = parseString(xml)

    # 保存xml
    image_name=image_name.split('/')[-1]
    save_xml = os.path.join(save_dir, image_name.replace('jpg', 'xml'))
    with open(save_xml, 'wb') as f:
        f.write(xml)
    return

## 1.2 开始生成voc格式xml

In [ ]:
import cv2
import os
train_base_path='dataset/Train'
# for i in range(10):
for i in range(len(train_data)):
    x=train_data.values[i,1]
    y=train_data.values[i,2]
    bbox=[[float(x-5), float(y-20), 40, 40]]
    # print(bbox)
    image_name=os.path.join(train_base_path,'JPEGImages', train_data.values[i,0])
    img=cv2.imread(image_name)
    # 0--height, 1--widht, 2--channel
    height, width, channel = img.shape
    
    save_xml(image_name, bbox, save_dir=os.path.join(train_base_path,'Annotations'), width=width, height=height, channel=channel)

738.96 770.54 40 40
1389.82 705.54 40 40
1356.74 850.72 40 40
698.15 722.44 40 40
1065.95 1017.54 40 40
1056.15 1003.47 40 40
1369.39 748.94 40 40
1787.11 1029.14 40 40
1081.53 969.92 40 40
1369.39 685.1 40 40
1374.5 559.92 40 40
-5.0 -20.0 40 40
837.77 675.04 40 40
979.73 994.21 40 40
1108.92 995.72 40 40
1004.14 955.72 40 40
1378.65 627.29 40 40
1048.16 1010.0999999999999 40 40
1424.32 685.04 40 40
1034.14 1015.1500000000001 40 40
1090.12 1013.98 40 40
1393.06 699.47 40 40
1002.9 991.25 40 40
1138.57 986.39 40 40
1028.55 1021.29 40 40
738.05 714.67 40 40
1072.09 1070.58 40 40
1055.48 986.33 40 40
1057.69 1069.17 40 40
1101.09 1008.44 40 40
1302.84 1029.57 40 40
1124.4 883.34 40 40
1112.15 1047.84 40 40
1011.49 1025.45 40 40
1115.18 996.51 40 40
1016.06 1070.37 40 40
1006.46 1005.56 40 40
1022.0899999999999 1118.73 40 40
1033.42 1093.73 40 40
1151.1 1082.33 40 40
1094.07 1098.22 40 40
1098.62 1076.61 40 40
1147.69 1030.83 40 40
1269.52 1145.12 40 40
1116.29 1147.23 40 40
1134.55 1078.

# 2.paddlex安装配置

## 2.1 paddlex安装

In [ ]:
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 13.7MB/s eta 0:00:01
     |████████████████████████████████| 215kB 34.6MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 9.0MB/s eta 0:00:01
     |████████████████████████████████| 102kB 23.5MB/s ta 0:00:01
     |████████████████████████████████| 153kB 22.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 21.7MB/s ta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278363 sha256=1c0d8c2843cc9da2ea642575c72610d89ecb45e2a8b722fb7653f31e56bd6d38
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


## 2.2 数据集划分

In [ ]:
# 分割训练集、测试集
!paddlex --split_dataset --format VOC --dataset_dir dataset/Train --val_value 0.2

Dataset Split Done.
Train samples: 640
Eval samples: 160
Test samples: 0
Split files saved in dataset/Train


# 3.paddlex数据集配置

## 3.1引入相关包

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import paddlex as pdx
from paddlex.det import transforms

## 3.2 Transform

In [ ]:
# 定义训练和验证时的transforms
# API说明 https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/seg_transforms.html
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32),
])

## 3.3 数据集

In [ ]:
# 定义训练和验证所用的数据集
# API说明：https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-changedetdataset
train_dataset = pdx.datasets.VOCDetection(
    data_dir='dataset/Train',
    file_list='dataset/Train/train_list.txt',
    label_list='dataset/Train/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.VOCDetection(
    data_dir='dataset/Train',
    file_list='dataset/Train/val_list.txt',
    label_list='dataset/Train/labels.txt',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


2021-05-31 08:31:28 [INFO]	Starting to read file list from dataset...
2021-05-31 08:31:29 [INFO]	640 samples in file dataset/Train/train_list.txt
creating index...
index created!
2021-05-31 08:31:29 [INFO]	Starting to read file list from dataset...
2021-05-31 08:31:29 [INFO]	160 samples in file dataset/Train/val_list.txt
creating index...
index created!


## 3.4 模型调用
注意不同模型，背景是否也算作一类，也是有说法的。

In [ ]:
# 初始化模型，并进行训练
# 可使用VisualDL查看训练指标，参考https://paddlex.readthedocs.io/zh_CN/develop/train/visualdl.html
num_classes = len(train_dataset.labels) + 1


# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#paddlex-det-yolov3
model = pdx.det.FasterRCNN(num_classes=num_classes)

# 4.开始训练
我也很无语，40*40的框，就这么点精度

```
2021-05-26 20:39:03 [INFO]	[EVAL] Finished, Epoch=17, bbox_map=0.002684 .
2021-05-26 20:39:07 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_17.
2021-05-26 20:39:07 [INFO]	Current evaluated best model in eval_dataset is epoch_8, bbox_map=4.625950541327286
```

In [18]:
# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#id1
# 各参数介绍与调整说明：https://paddlex.readthedocs.io/zh_CN/develop/appendix/parameters.html
model.train(
    num_epochs=100,
    train_dataset=train_dataset,
    train_batch_size=26,
    eval_dataset=eval_dataset,
    learning_rate=0.01,
    lr_decay_epochs=[210, 240],
    save_dir='output/faster_rcnn_r50_fpn',    
    use_vdl=True)

100%|██████████| 160/160 [00:35<00:00,  4.48it/s]


2021-05-31 09:22:22 [INFO]	[EVAL] Finished, Epoch=22, bbox_map=0.005938 .
2021-05-31 09:22:27 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_22.
2021-05-31 09:22:27 [INFO]	Current evaluated best model in eval_dataset is epoch_14, bbox_map=16.76804549119889
2021-05-31 09:22:44 [INFO]	[TRAIN] Epoch=23/100, Step=2/24, loss=0.154241, loss_cls=0.090258, loss_bbox=0.032308, loss_rpn_cls=0.023144, loss_rpn_bbox=0.008532, lr=0.01, time_each_step=4.09s, eta=2:56:10
2021-05-31 09:22:50 [INFO]	[TRAIN] Epoch=23/100, Step=4/24, loss=0.157992, loss_cls=0.087023, loss_bbox=0.035282, loss_rpn_cls=0.026479, loss_rpn_bbox=0.009207, lr=0.01, time_each_step=3.99s, eta=2:56:0
2021-05-31 09:22:57 [INFO]	[TRAIN] Epoch=23/100, Step=6/24, loss=0.160215, loss_cls=0.09518, loss_bbox=0.037092, loss_rpn_cls=0.019536, loss_rpn_bbox=0.008406, lr=0.01, time_each_step=4.03s, eta=2:55:52
2021-05-31 09:23:04 [INFO]	[TRAIN] Epoch=23/100, Step=8/24, loss=0.209895, loss_cls=0.124031, loss_bbox=0.062399, loss_rpn_cl

100%|██████████| 160/160 [00:35<00:00,  4.48it/s]


2021-05-31 09:24:38 [INFO]	[EVAL] Finished, Epoch=23, bbox_map=0.001765 .
2021-05-31 09:24:43 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_23.
2021-05-31 09:24:43 [INFO]	Current evaluated best model in eval_dataset is epoch_14, bbox_map=16.76804549119889
2021-05-31 09:24:59 [INFO]	[TRAIN] Epoch=24/100, Step=2/24, loss=0.172263, loss_cls=0.09959, loss_bbox=0.040764, loss_rpn_cls=0.023301, loss_rpn_bbox=0.008607, lr=0.01, time_each_step=4.05s, eta=2:54:18
2021-05-31 09:25:05 [INFO]	[TRAIN] Epoch=24/100, Step=4/24, loss=0.159202, loss_cls=0.075994, loss_bbox=0.032077, loss_rpn_cls=0.043613, loss_rpn_bbox=0.007517, lr=0.01, time_each_step=4.0s, eta=2:54:9
2021-05-31 09:25:13 [INFO]	[TRAIN] Epoch=24/100, Step=6/24, loss=0.195236, loss_cls=0.117686, loss_bbox=0.040705, loss_rpn_cls=0.026133, loss_rpn_bbox=0.010712, lr=0.01, time_each_step=4.06s, eta=2:54:2
2021-05-31 09:25:19 [INFO]	[TRAIN] Epoch=24/100, Step=8/24, loss=0.227339, loss_cls=0.116317, loss_bbox=0.050626, loss_rpn_cls=

100%|██████████| 160/160 [00:34<00:00,  4.59it/s]


...

100%|██████████| 160/160 [00:35<00:00,  4.46it/s]


2021-05-31 10:46:31 [INFO]	[EVAL] Finished, Epoch=59, bbox_map=0.00366 .
2021-05-31 10:46:35 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_59.
2021-05-31 10:46:35 [INFO]	Current evaluated best model in eval_dataset is epoch_47, bbox_map=30.206878346987164
2021-05-31 10:46:50 [INFO]	[TRAIN] Epoch=60/100, Step=2/24, loss=0.214874, loss_cls=0.126142, loss_bbox=0.06047, loss_rpn_cls=0.02298, loss_rpn_bbox=0.005282, lr=0.01, time_each_step=4.03s, eta=1:32:19
2021-05-31 10:46:57 [INFO]	[TRAIN] Epoch=60/100, Step=4/24, loss=0.240932, loss_cls=0.107212, loss_bbox=0.029198, loss_rpn_cls=0.097506, loss_rpn_bbox=0.007016, lr=0.01, time_each_step=3.99s, eta=1:32:10
2021-05-31 10:47:03 [INFO]	[TRAIN] Epoch=60/100, Step=6/24, loss=0.347694, loss_cls=0.108408, loss_bbox=0.026014, loss_rpn_cls=0.204864, loss_rpn_bbox=0.008408, lr=0.01, time_each_step=3.95s, eta=1:32:1
2021-05-31 10:47:11 [INFO]	[TRAIN] Epoch=60/100, Step=8/24, loss=0.294609, loss_cls=0.143318, loss_bbox=0.065179, loss_rpn_cls

Process Process-951:
Process Process-946:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/reader.py:1294: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn('Your reader has raised an exception!')
[WARNING 2021-05-31 10:48:06,536 reader.py:1294] Your reader has raised an exception!
Process Process-949:
Process Process-952:
Process Process-945:
Process Process-948:
Process Process-950:
Process Process-947:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/transforms/det_transforms.py", line 115, in decode_image
    cv2.IMREAD_ANYCOLOR | cv2.IMREAD_COLOR)


# 5.模型预测

## 5.1 先预测一张

In [21]:
import paddlex as pdx
model = pdx.load_model('output/faster_rcnn_r50_fpn/best_model')
image_name = 'dataset/PALM-Testing400-Images/T0001.jpg'
result = model.predict(image_name)
print(result)
pdx.det.visualize(image_name, result, threshold=0.5, save_dir='./output/faster_rcnn_r50_fpn')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:299: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/fpn.py:172
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:299: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/bbox_head.py:278
The behavior of expression A / B has been unified with elementwise_div(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwi

2021-05-31 10:48:47 [INFO]	Model[FasterRCNN] loaded.
[]
2021-05-31 10:48:48 [INFO]	The visualized result is saved as ./output/faster_rcnn_r50_fpn/visualize_T0001.jpg


In [24]:
bbox=result[0]['bbox']
print(bbox)

IndexError: list index out of range

## 5.2 生成文件名列表

In [25]:
# 预测数据集val_list
val_list=[]
for i in range(1,401,1):
    filename='T'+ str(i).zfill(4)+'.jpg'
    # print(filename)
    val_list.append(filename+'\n')

with open('val_list.txt','w') as f:
    f.writelines(val_list)

In [26]:
pd_A=[]
with open('val_list.txt', 'r') as f:
    for line in f:
        line='dataset/PALM-Testing400-Images/'+line
        pd_A.append(line.split('\n')[0])

## 5.3预测并生成x/y坐标

In [ ]:
import paddlex as pdx

model = pdx.load_model('output/faster_rcnn_r50_fpn/best_model')
# x坐标
pd_B=[]
# y坐标
pd_C=[]
for item in pd_A :
    result = model.predict(item)
    if  result:
        bbox=result[0]['bbox']
        x0=bbox[0]
        x1=bbox[2]
        y0=bbox[1]
        y1=bbox[3]
        x=0.5*(x1+x0)
        y=0.5*(y1+y0)
    else:
        x=0
        y=0
    pd_B.append(x)
    pd_C.append(y)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:299: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/fpn.py:172
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:299: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/bbox_head.py:278
The behavior of expression A / B has been unified with elementwise_div(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwi

2021-05-31 10:50:39 [INFO]	Model[FasterRCNN] loaded.


## 5.4 构造pandas framework

In [ ]:
# 文件名列
pd_A=[]
with open('val_list.txt', 'r') as f:
    for line in f:
        pd_A.append(line.split('\n')[0])

In [ ]:
# 构造pandas的DataFrame
df= pd.DataFrame({'FileName': pd_A, 'Fovea_x':pd_B, 'Fovea_y':pd_C})

## 5.5 保存结果

In [ ]:
# 保存为提交文件
df.to_csv("Fovea_Localization_Results.csv", index=None)

In [ ]:
# 读取检查下
data=pd.read_csv('Fovea_Localization_Results.csv')

In [ ]:
# 查看下生成数据
data.head(20)